In [25]:
import cv2
from sklearn.cluster import KMeans
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import pandas as pd
import os
from math import floor,sqrt
from sklearn.preprocessing import StandardScaler
from moviepy.editor import VideoFileClip
import librosa

### Annotations loader

In [26]:
# Load annotations
def load_annotations(anno_file, info_file):
    # Read the annotation and info files
    annotations = pd.read_csv(anno_file, sep='\t', header=None)
    info = pd.read_csv(info_file, sep='\t', header=None)
    
    # Rename columns for better understanding
    annotations.columns = ['video_id', 'category', 'importance_score']
    info.columns = ['category_code', 'video_id', 'title', 'url', 'length']
    
    return annotations, info

### Preprocessing and Feature Extraction


#### a. Video Processing and Frame Extraction


In [27]:
def extract_frames(video_path, frame_rate=25):
    video = cv2.VideoCapture(video_path)
    count = 0
    success = True
    frames = []
    
    while success:
        success, image = video.read()
        if count % frame_rate == 0 and success:
            frames.append(image)
        count += 1

    video.release()
    return frames

#### b. Audio Extraction


In [28]:

def extract_audio_from_video(video_path, output_audio_path):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(output_audio_path)
    video.close()

#### c.Audio Features

In [29]:
def extract_audio_features(audio_path):
    """
    Extract audio features from an audio file, specifically MFCCs.

    :param audio_path: Path to the audio file.
    :return: Array of MFCCs.
    """
    # Load the audio file
    y, sr = librosa.load(audio_path)

    # Extract MFCCs from the audio
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # You can change n_mfcc based on your needs

    # To capture the variation over time, you might compute statistics across MFCCs over time
    # Here, we compute the mean of the MFCCs across time
    mfccs_processed = np.mean(mfccs.T,axis=0)

    return mfccs_processed

#### d. Feature Extraction (example with visual features using a CNN)


In [30]:
model = VGG16(weights='imagenet', include_top=False)

def extract_visual_features(frames):
    features = []
    for frame in frames:
        if frame is not None:
            img = cv2.resize(frame, (224, 224))  # Resize frame to 224x224
            img = img_to_array(img)        # Convert to array
            img = np.expand_dims(img, axis=0)    # Add batch dimension
            img = preprocess_input(img)          # Preprocess for VGG16
            
            feature = model.predict(img,use_multiprocessing=True,workers=4)
            features.append(feature.flatten())

    return features

### Unsupervised Learning Model


In [31]:
def cluster_features(features, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(features)
    return kmeans.labels_

### Connect Audio/Annotation/Video

In [32]:
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from math import floor, sqrt

def integrate_and_cluster_features(video_path, anno_file, info_file, num_clusters=1):
    """
    Integrate visual, audio, and annotation features from a video,
    and perform clustering on the combined features.

    :param video_path: Path to the video file.
    :param anno_file: Path to the annotation file.
    :param info_file: Path to the info file.
    :param num_clusters: Number of clusters to use in KMeans.
    :return: Cluster labels for each data point.
    """
    # Extract frames from the video
    frames = extract_frames(video_path)  # Ensure this function is defined

    # Extract visual features
    visual_features = extract_visual_features(frames)  # Ensure this function is defined

    # Extract audio
    audio_output_path = 'extracted_audio.wav'
    extract_audio_from_video(video_path, audio_output_path)  # Ensure this function is defined

    # Extract audio features
    audio_features = extract_audio_features(audio_output_path)  # Ensure this function is defined

    # Load annotations
    annotations, info = load_annotations(anno_file, info_file)  # Ensure this function is defined

    # If annotations include categorical data, convert it to numerical format
    label_encoder = LabelEncoder()
    categorical_columns = ['category']  # Replace with your actual categorical column names
    for column in categorical_columns:
        if column in annotations.columns:
            annotations[column] = label_encoder.fit_transform(annotations[column])

    # Exclude non-numerical data from annotations if necessary
    annotation_features = annotations.drop(columns=['video_id']).values

    # Combine features
    combined_features = []
    for i, frame in enumerate(frames):
        # Concatenate visual, audio, and annotation features
        # Ensure that all features are NumPy arrays or lists of floats
        annotation_list=[]
        for annotation in annotation_features:
                annotation_list.append(annotation)
        combined_feature = np.concatenate([
            np.array(visual_features[i], dtype=float),
            np.array(audio_features, dtype=float),
            np.array(annotation_list, dtype=float)
        ])
        combined_features.append(combined_feature)

    # Convert the list of arrays to a 2D NumPy array
    combined_features_array = np.array(combined_features)

    # Normalize the features
    combined_features_normalized = StandardScaler().fit_transform(combined_features_array)

    # Calculate the number of clusters if not provided
    if num_clusters is None:
        num_clusters = floor(sqrt(len(combined_features_normalized)/1.3))

    # Perform clustering
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(combined_features_normalized)

    return [frames , kmeans.labels_]
    # return the frames and the labels


### Summarization

In [33]:
def create_summary(frames, labels, cluster_to_use):
    summary_frames = [frame for frame, label in zip(frames, labels) if label == cluster_to_use]
    print(f"Number of frames selected for summary: {len(summary_frames)}")
    return summary_frames

#### Video Creator

In [34]:
def create_video_from_frames(frames, output_path, frame_rate=25):
    if not frames:
        print("No frames to create a video.")
        return 
    # Determine the width and height from the first frame
    height, width, layers = frames[0].shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID' if 'mp4v' doesn't work
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (width, height))

    # Write each frame to the video
    for frame in frames:
        out.write(frame)

    # Release the VideoWriter object
    out.release()

# Code to create summary video

#### Variables

In [35]:
annotation_path='datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-anno.tsv'
info_path='datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-info.tsv'

In [36]:
video_path='datasets/ydata-tvsum50-v1_1/video/'
summary_video_path='datasets/summary_videos/'

##### Get the list of the videos in the folder

In [37]:
video_list = [video for video in os.listdir(video_path) if video.endswith('.mp4')]  # List comprehension

#### Function for videoSummarizion

In [38]:
def videoSummarize(annotation_path=None, info_path=None, video_path=None, summary_video_path=None,video_list=None):
    for video in video_list:  
        frames,labels = integrate_and_cluster_features(video_path+video, annotation_path, info_path)
        
        print(labels)
        # Select Representative Frames: Cluster 2
        summary_frames = create_summary(frames, labels, cluster_to_use=2)  # example for one cluster
        
        # Create Summary Video
        create_video_from_frames(summary_frames, summary_video_path+video)
        
        # Video summarization for cluster 2 

#### Code for dynamic annotation and video summarization

In [39]:
videoSummarize(annotation_path, info_path, video_path, summary_video_path, video_list)

1/1 [==============================] - 0s 152ms/step
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.


ValueError: could not convert string to float: '4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,